In [2]:
import pandas as pd
from py_files import data_manager
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from pgmpy.models import BayesianNetwork, BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator, ExpectationMaximization
from pgmpy.inference import VariableElimination

/Users/jasonvasquez/opt/anaconda3/envs/py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#load data
df = pd.read_csv('data/play_by_play_2018_19.csv', encoding='latin1')

/var/folders/54/zvdnrgtn2fb_bdky24ktf8jc0000gn/T/ipykernel_19806/356840926.py:2: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/play_by_play_2018_19.csv', encoding='latin1')


In [4]:
df = df.dropna(subset=['event_team_type'])
    
# Get all unique game ids
unique_game_ids = df.game_id.unique()

# Get the label encoder for the teams
team_names = np.sort(df.event_team.dropna().unique())
label_encoder = data_manager.get_label_encoder(team_names)

# Label encode the teams
df['team_encoded'] = label_encoder.transform(df.event_team)

df.head()

/Users/jasonvasquez/opt/anaconda3/envs/py311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/jasonvasquez/opt/anaconda3/envs/py311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


,xg,event_id,event_type,event,secondary_type,event_team,event_team_type,description,period,period_seconds,...,home_conference_name,home_id,away_name,away_abbreviation,away_division_name,away_division_name_short,away_conference_name,away_id,venue_id,team_encoded
1,NaN,2.018020e+13,CHANGE,Change,NaN,Montréal Canadiens,away,"ON: Carey Price, Jeff Petry, Paul Byron, Arttu...",1,0,...,Eastern,10,Montréal Canadiens,MTL,Atlantic,ATL,Eastern,8,NaN,15
2,NaN,2.018020e+13,CHANGE,Change,Line change,Toronto Maple Leafs,home,"ON: Patrick Marleau, Jake Gardiner, Tyler Enni...",1,0,...,Eastern,10,Montréal Canadiens,MTL,Atlantic,ATL,Eastern,8,NaN,26
3,NaN,2.018020e+13,FACEOFF,Faceoff,NaN,Montréal Canadiens,away,Max Domi faceoff won against Auston Matthews,1,0,...,Eastern,10,Montréal Canadiens,MTL,Atlantic,ATL,Eastern,8,NaN,15
4,0.019187,2.018020e+13,SHOT,Shot,Backhand,Montréal Canadiens,away,Artturi Lehkonen Backhand saved by Frederik An...,1,29,...,Eastern,10,Montréal Canadiens,MTL,Atlantic,ATL,Eastern,8,NaN,15
5,NaN,2.018020e+13,CHANGE,Change,On the fly,Montréal Canadiens,away,"ON: Tomas Tatar, Brendan Gallagher; OFF: Paul ...",1,38,...,Eastern,10,Montréal Canadiens,MTL,Atlantic,ATL,Eastern,8,NaN,15


In [5]:
def get_data(df, game_id):
    """
    Extracts and organizes event data for a specific game.

    Args:
    df (DataFrame): The DataFrame containing the game data.
    game_id (int): The ID of the game to retrieve data for.

    Returns:
    tuple: A tuple containing two DataFrames, one for home team events and one for away team events.
    """

    # Filter the DataFrame to get data for the specified game and drop rows with NaN values in event_team_type
    game1 = df[df.game_id == game_id] 
    game1 = game1.dropna(subset=['event_team_type'])

    # Get unique event types (excluding CHANGE)
    event_types = game1.event_type.unique()
    event_types = np.delete(event_types, np.where(event_types == 'CHANGE'))
    event_types = np.append(event_types, ['TIME_REMAINING', 'HOME', 'WIN', 'TEAM', 'GAME_ID', 'CORSI', 'FENWICK'])

    # Create dictionaries to store event counts for home and away teams
    final_dict = {f'HOME_{event}': 0 for event in event_types}
    away_dict = {f'AWAY_{event}': 0 for event in event_types}
    final_dict.update(away_dict)

    # Create DataFrames to store event data for home and away teams
    final_df = False

    # Iterate through the events in the game and count them
    for _, row in game1.iterrows():
        # Skip events with NaN event_team_type or events of type CHANGE
        if pd.isnull(row['event_team_type']) or row['event_type'] == 'CHANGE':
            continue
        
        # Determine if the event belongs to the home or away team and update counts accordingly
        if row['event_team_type'] == 'home':
            final_dict[f"HOME_{row['event_type']}"] += 1
            final_dict['TIME_REMAINING'] = row['game_seconds_remaining']
            final_dict['HOME_HOME'] = 1
            final_dict['WIN'] = 1 if row['home_final'] > row['away_final'] else 0
            final_dict['HOME_TEAM'] = row['team_encoded']
            final_dict['GAME_ID'] = game_id

            # Get the Corsi and Fenwick for the home team
            if row['strength_code'] == "EV":
                final_dict['HOME_CORSI'] = calculate_corsi(final_dict, 'HOME')
                final_dict['HOME_FENWICK'] = calculate_fenwick(final_dict, 'HOME')
                final_dict['HOME_CORSI_FOR'] = calculate_corsi_for(final_dict['HOME_CORSI'], final_dict['AWAY_CORSI'])
                final_dict['HOME_FENWICK_FOR'] = calculate_fenwick_for(final_dict['HOME_FENWICK'], final_dict['AWAY_FENWICK'])
            
            # home_df = home_df.append(home_dict, ignore_index=True)
        else:
            final_dict[f"AWAY_{row['event_type']}"] += 1
            final_dict['TIME_REMAINING'] = row['game_seconds_remaining']
            final_dict['AWAY_HOME'] = 0
            final_dict['WIN'] = 0 if row['home_final'] < row['away_final'] else 1
            final_dict['AWAY_TEAM'] = row['team_encoded']
            final_dict['GAME_ID'] = game_id
            
            # Get the Corsi and Fenwick for the away team
            if row['strength_code'] == "EV":
                final_dict['AWAY_CORSI'] = calculate_corsi(final_dict, "AWAY")
                final_dict['AWAY_FENWICK'] = calculate_fenwick(final_dict, "AWAY")
                final_dict['AWAY_CORSI_FOR'] = calculate_corsi_for(final_dict['AWAY_CORSI'], final_dict['HOME_CORSI'])
                final_dict['AWAY_FENWICK_FOR'] = calculate_fenwick_for(final_dict['AWAY_FENWICK'], final_dict['HOME_FENWICK'])
                
            # away_df = away_df.append(away_dict, ignore_index=True)
        if type(final_df) is bool:
            # print(final_dict)
            final_df = pd.DataFrame(final_dict, index=[0])
        else:
            append_row = pd.DataFrame(final_dict, index=[0])
            final_df = pd.concat([final_df, append_row], ignore_index=True)
        
    return final_df



def get_label_encoder(teams):
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    le.fit(teams)
    return le


def calculate_corsi(team_dict, h_or_a):
    """
    Calculates the Corsi for a team.

    Args:
    team_dict (dict): A dictionary containing event counts for a team.

    Returns:
    int: The Corsi for the team.
    """
    return team_dict[f'{h_or_a}_SHOT'] + team_dict[f'{h_or_a}_MISSED_SHOT'] + team_dict[f'{h_or_a}_BLOCKED_SHOT']


def calculate_fenwick(team_dict, h_or_a):
    """
    Calculates the Fenwick for a team.

    Args:
    team_dict (dict): A dictionary containing event counts for a team.

    Returns:
    int: The Fenwick for the team.
    """
    return team_dict[f'{h_or_a}_SHOT'] + team_dict[f'{h_or_a}_MISSED_SHOT']


def calculate_corsi_for(team_corsi, opp_corsi):
    """
    Calculates the Corsi For Percentage for a team.

    Args:
    team_corsi (int): The Corsi for the team.
    opp_corsi (int): The Corsi for the opposing team.

    Returns:
    float: The Corsi For Percentage for the team.
    """
    # Check if the denominator is zero to avoid division by zero
    if team_corsi + opp_corsi == 0:
        return 0
    
    return team_corsi / (team_corsi + opp_corsi)


def calculate_fenwick_for(team_fenwick, opp_fenwick):
    """
    Calculates the Fenwick For Percentage for a team.

    Args:
    team_fenwick (int): The Fenwick for the team.
    opp_fenwick (int): The Fenwick for the opposing team.

    Returns:
    float: The Fenwick For Percentage for the team.
    """
    # Check if the denominator is zero to avoid division by zero
    if team_fenwick + opp_fenwick == 0:
        return 0
    
    return team_fenwick / (team_fenwick + opp_fenwick)

def merge_home_away(home_df, away_df):
     # change the names in the home_df to be HOME_[current column name] and same for away_df
     home_df.columns = ['HOME_' + col for col in home_df.columns]
     away_df.columns = ['AWAY_' + col for col in away_df.columns]
     # change TIME_REMAINING, WIN, GAME_ID to be the same in both dataframes
     home_df = home_df.rename(columns={'HOME_TIME_REMAINING': 'TIME_REMAINING', 'HOME_WIN': 'WIN', 'HOME_GAME_ID': 'GAME_ID'})
     away_df = away_df.rename(columns={'AWAY_TIME_REMAINING': 'TIME_REMAINING', 'AWAY_WIN': 'WIN', 'AWAY_GAME_ID': 'GAME_ID'})

     full = pd.merge(home_df, away_df, on=['TIME_REMAINING', 'WIN', 'GAME_ID'], how='outer')

     # if home_df has 1 in WIN column than make all the values in the WIN column 1
     if 1 in home_df.WIN.unique():
          full.WIN = 1
     elif 1 in away_df.WIN.unique():
          full.WIN = 0

     full.drop(columns=['HOME_HOME', 'AWAY_HOME'])
     full = full.sort_values(by='TIME_REMAINING')
     full.fillna(method='ffill', inplace=True)
     full.fillna(0, inplace=True)

     return full

In [6]:
# Copy the dataframe
df_copy = df.copy()

# Drop NaN's from event_team_type
df_copy = df_copy.dropna(subset=['event_team'])

# Get all of the data
unique_game_ids = df.game_id.unique()

# Get the label encoder for the teams
team_names = np.sort(df_copy.event_team.dropna().unique())
label_encoder = get_label_encoder(team_names)

# Label encode the teams
df_copy['team_encoded'] = label_encoder.transform(df_copy.event_team)

# Iterate through the games
game_bar = tqdm(total=len(unique_game_ids))
final_df = pd.DataFrame()
for game_id in unique_game_ids:
    final_df_p = get_data(df_copy, game_id)
    final_df = pd.concat([final_df, final_df_p], ignore_index=True)
    game_bar.update(1)

/Users/jasonvasquez/opt/anaconda3/envs/py311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/jasonvasquez/opt/anaconda3/envs/py311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
100%|██████████| 1358/1358 [07:56<00:00,  2.44it/s]

In [7]:
label_column = 'WIN'
df = final_df.copy()

X = df.drop(columns=[label_column, 'HOME_GAME_ID', 'AWAY_GAME_ID', 'HOME_TIME_REMAINING', 'AWAY_TIME_REMAINING', 'HOME_WIN', 'AWAY_WIN'])
y = df[label_column]

game_groups = df.groupby('GAME_ID')

game_ids = df.GAME_ID.unique()

In [8]:
# Define X and y (features and target)
X = df.drop(['WIN', 'GAME_ID'], axis=1)  # Assuming 'WIN' is the target variable
y = df['WIN']  # Keep y as a pandas Series

# Ensure y contains only binary classes (0 and 1)
y_unique = y.unique()
if len(y_unique) != 2 or 0 not in y_unique or 1 not in y_unique:
    raise ValueError("Target variable y must contain binary classes (0 and 1)")

# Split the dataset into training and testing sets

# Instantiate and train the exponential smoothing model
model = ExponentialSmoothing(X, trend='add', damped_trend=True)
fitted_model = model.fit()
break

# Extract data for a certain timestep from your dataset
game_id = df['GAME_ID'].unique()[2]  # Specify the game ID
desired_timestep = 1500  # Specify the desired timestep
game_data = df[df['GAME_ID'] == game_id]
timestep_data = game_data[game_data['TIME_REMAINING'] < desired_timestep]

# Prepare the timestep data for prediction
X_timestep = timestep_data.drop(['WIN', 'GAME_ID'], axis=1)



# Predict the outcome for the timestep
outcome = fitted_model.predict(start=len(y), end=len(y) + len(X_timestep) - 1)

# Assuming binary outcome (0 or 1)
outcome_binary = (outcome > 0.5).astype(int)

# get final prediction of outcome binary

# Print the predicted outcome
print("Predicted outcome:", 'Home team win')


ValueError: endog is required to have ndim 1 but has ndim 2

In [9]:
final_df.to_hdf('data/processed_data.h5', key='df', mode='w')

In [ ]:
final_df = pd.read_hdf('data/processed_data.h5', key='df')

In [10]:
label_column = 'WIN'
df = final_df.copy()

X = df.drop(columns=[label_column, 'HOME_GAME_ID', 'AWAY_GAME_ID', 'HOME_TIME_REMAINING', 'AWAY_TIME_REMAINING'])
y = df[label_column]

game_groups = df.groupby('GAME_ID')

game_ids = df.GAME_ID.unique()

train_ids, test_ids = train_test_split(game_ids, test_size=0.2, random_state=42)

train = df[df.GAME_ID.isin(train_ids)]
test = df[df.GAME_ID.isin(test_ids)]

train_X = train.drop(columns=[label_column])
train_y = train[label_column]
test_X = test.drop(columns=[label_column])
test_y = test[label_column]

In [29]:
keepers = [
    "HOME_GOAL",
    "AWAY_GOAL",
    "HOME_SHOT",
    "AWAY_SHOT",
    "HOME_MISSED_SHOT",
    "AWAY_MISSED_SHOT",
    "HOME_BLOCKED_SHOT",
    "AWAY_BLOCKED_SHOT",
    "HOME_FACEOFF",
    "AWAY_FACEOFF",
    "TIME_REMAINING",
    "WIN",
]

train = train[keepers]
test = test[keepers]

# take 10% of train
train2 = train.sample(frac=0.1, random_state=42)

In [40]:
keepers = [
    "HOME_GOAL",
    "AWAY_GOAL",
    "TIME_REMAINING",
    "WIN",
]

train = train[keepers]
test = test[keepers]

# take 10% of train
train2 = train

model_dict = [(feature, 'WIN') for feature in train.columns if feature != 'WIN']

# Define the structure of the Bayesian network
model = BayesianNetwork(model_dict)

# Learn the parameters of the Bayesian network using Maximum Likelihood Estimation
model.fit(train2, estimator=ExpectationMaximization)

# Create an inference object for probabilistic inference
inference = VariableElimination(model)

evidence = {
    'HOME_GOAL': 0,
    'AWAY_GOAL': 0,
    'TIME_REMAINING': 3600
}

# Perform inference to compute posterior probabilities
posterior_probabilities = inference.query(variables=['WIN'], evidence=evidence)

# Extract probabilities of each outcome
home_team_win_probability = posterior_probabilities.values[1]
away_team_win_probability = posterior_probabilities.values[0]

print("Probability of Home Team Win:", home_team_win_probability)
print("Probability of Away Team Win:", away_team_win_probability)

  0%|          | 0/100 [02:08<?, ?it/s]

Probability of Home Team Win: 0.5413602941176471
Probability of Away Team Win: 0.4586397058823529


In [45]:
inference = VariableElimination(model)

evidence = {
    'HOME_GOAL': 3,
    'AWAY_GOAL': 0,
}

# Perform inference to compute posterior probabilities
posterior_probabilities = inference.query(variables=['WIN'], evidence=evidence)

# Extract probabilities of each outcome
home_team_win_probability = posterior_probabilities.values[1]
away_team_win_probability = posterior_probabilities.values[0]

print("Probability of Home Team Win:", home_team_win_probability)
print("Probability of Away Team Win:", away_team_win_probability)

Probability of Home Team Win: 0.8174991249864597
Probability of Away Team Win: 0.18250087501354026
